<a href="https://colab.research.google.com/github/yadavrahyl2k15/ASL-SIGN_RECOGITION/blob/main/Sign_language.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import os
import random
import shutil
import itertools
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.losses import CategoricalCrossentropy

In [19]:
os.chdir("/content/drive/MyDrive/Dataset")
if os.path.isdir("train/0/") is False:
  os.mkdir("train")
  os.mkdir("valid")
  os.mkdir("test")
  for i in range(2,10):
    shutil.move(f'{i}','train/')
    os.mkdir(f'valid/{i}')
    os.mkdir(f'test/{i}')
    
  valid_samples=random.sample(os.listdir(f'train/{i}'),30)
  for j in valid_samples:
    shutil.move(f'train/{i}/{j}',f"valid/{i}")
    
  test_samples=random.sample(os.listdir(f"train/{i}"),5)
  for k in test_samples:
    shutil.move(f"train/{i}/{k}",f"test/{i}")
os.chdir("../..")


In [20]:

train_path="/content/drive/MyDrive/Dataset/train"
valid_path="/content/drive/MyDrive/Dataset/valid"
test_path='/content/drive/MyDrive/Dataset/test'

In [21]:
train_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=train_path,target_size=(224,224),batch_size=10)
valid_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=valid_path,target_size=(224,224),batch_size=10)
test_batches=ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=test_path,target_size=(224,224),batch_size=10,shuffle=False)

Found 1991 images belonging to 10 classes.
Found 36 images belonging to 10 classes.
Found 35 images belonging to 10 classes.


MODIFY MODEL

In [22]:
mobile=tf.keras.applications.mobilenet.MobileNet()

In [23]:
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32

In [24]:
x=mobile.layers[-6].output
output=Dense(units=10,activation="softmax")(x)

In [25]:
model=Model(inputs=mobile.input,outputs=output)
for layers in model.layers[:-23]:
  layers.trainable=False

In [26]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0   

TRAIN THE MODEL


In [27]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss="CategoricalCrossentropy",metrics=["Accuracy"])
model.fit(x=train_batches,validation_data=valid_batches,epochs=10,verbose=2)

Epoch 1/10
200/200 - 139s - loss: 0.6046 - accuracy: 0.0000e+00 - val_loss: 4.9147 - val_accuracy: 0.0000e+00
Epoch 2/10
200/200 - 138s - loss: 0.0984 - accuracy: 0.0000e+00 - val_loss: 6.8367 - val_accuracy: 0.0000e+00
Epoch 3/10
200/200 - 137s - loss: 0.0575 - accuracy: 0.0000e+00 - val_loss: 7.3103 - val_accuracy: 0.0000e+00
Epoch 4/10
200/200 - 138s - loss: 0.0352 - accuracy: 0.0000e+00 - val_loss: 6.0247 - val_accuracy: 0.0000e+00
Epoch 5/10
200/200 - 134s - loss: 0.0277 - accuracy: 0.0000e+00 - val_loss: 7.8231 - val_accuracy: 0.0000e+00
Epoch 6/10
200/200 - 132s - loss: 0.0238 - accuracy: 0.0000e+00 - val_loss: 8.1335 - val_accuracy: 0.0000e+00
Epoch 7/10
200/200 - 132s - loss: 0.0159 - accuracy: 0.0000e+00 - val_loss: 6.4900 - val_accuracy: 0.0000e+00
Epoch 8/10
200/200 - 133s - loss: 0.0121 - accuracy: 0.0000e+00 - val_loss: 8.6457 - val_accuracy: 0.0000e+00
Epoch 9/10
200/200 - 139s - loss: 0.0093 - accuracy: 0.0000e+00 - val_loss: 7.9925 - val_accuracy: 0.0000e+00
Epoch 10/1

PREDICT MODEL

In [30]:
from sklearn.metrics import confusion_matrix
test_labels=test_batches.classes
predictions=model.predict(x=test_batches,verbose=0)

cm=confusion_matrix(y_true=test_labels,y_pred=predictions.argmax(axis=1))
test_batches.class_indices

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9}

In [31]:
cm

array([[30,  0,  0],
       [ 0,  0,  0],
       [ 0,  5,  0]])